In [2]:
!pip install ultralytics opencv-python-headless deep_sort_realtime

In [3]:
import cv2
import torch
import numpy as np
import os
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from collections import defaultdict

In [4]:
video_path = '/content/15sec_input_720p.mp4'
model_path = "/content/best.pt"
output_path = "/content/output_with_ids.mp4"

In [5]:
from google.colab.patches import cv2_imshow

In [6]:
input_path="/content/15sec_input_720p.mp4"

In [7]:
from google.colab.patches import cv2_imshow

In [8]:
CONFIG = {
    'model_path': '/content/best.pt',
    'video_path': '/content/15sec_input_720p.mp4',
    'output_video_path': '/content/output_with_ids.mp4',
    'temp_frames_dir': 'temp_frames',
    'conf_threshold': 0.5,
    'iou_threshold': 0.4,
    'max_age': 60,
    'n_init': 5,
    'max_cosine_distance': 0.4,
    'frame_skip': 0,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'show_output': True
}

class PlayerTracker:
    def __init__(self, config):
        self.config = config
        self.model = YOLO(config['model_path']).to(config['device'])
        self.tracker = DeepSort(
            max_age=config['max_age'],
            n_init=config['n_init'],
            max_cosine_distance=config['max_cosine_distance'],
            nn_budget=None,
            embedder="mobilenet",
            half=True,
            embedder_gpu=True
        )
        self.id_colors = {}
        self.frame_buffer = []
        os.makedirs(config['temp_frames_dir'], exist_ok=True)

    def get_color(self, track_id):
        if track_id not in self.id_colors:
            self.id_colors[track_id] = tuple(np.random.randint(0, 255, 3).tolist())
        return self.id_colors[track_id]

    def process_video(self):
        cap = cv2.VideoCapture(self.config['video_path'])
        if not cap.isOpened():
            print("Error: Could not open video.")
            return False
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        print(f"\nProcessing video: {self.config['video_path']}")
        print(f"Resolution: {frame_width}x{frame_height} at {fps:.2f} FPS")
        print(f"Total frames: {total_frames}")
        print(f"Using device: {self.config['device']}\n")

        frame_count = 0
        processed_count = 0
        processing_times = []

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1
            if self.config['frame_skip'] > 0 and frame_count % (self.config['frame_skip'] + 1) != 0:
                continue

            start_time = cv2.getTickCount()

            results = self.model.predict(
                frame,
                conf=self.config['conf_threshold'],
                iou=self.config['iou_threshold'],
                device=self.config['device'],
                verbose=False
            )

            boxes = results[0].boxes.xyxy.cpu().numpy() if results[0].boxes is not None else []
            confs = results[0].boxes.conf.cpu().numpy() if results[0].boxes is not None else []
            class_ids = results[0].boxes.cls.cpu().numpy() if results[0].boxes is not None else []

            detections = []
            for box, conf, class_id in zip(boxes, confs, class_ids):
                x1, y1, x2, y2 = box
                w, h = x2 - x1, y2 - y1
                detections.append(([x1, y1, w, h], conf, int(class_id)))

            tracks = self.tracker.update_tracks(detections, frame=frame)

            for track in tracks:
                if not track.is_confirmed():
                    continue

                track_id = str(track.track_id)
                l, t, w, h = track.to_ltrb()
                bbox = tuple(map(int, (l, t, w, h)))
                color = self.get_color(track_id)

                cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[0]+bbox[2], bbox[1]+bbox[3]), color, 2)
                cv2.putText(frame, f'ID: {track_id}', (bbox[0], bbox[1]-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

            processing_time = (cv2.getTickCount() - start_time) / cv2.getTickFrequency()
            processing_times.append(processing_time)

            fps_text = f'FPS: {1/processing_time:.2f}' if processing_time > 0 else 'FPS: -'
            cv2.putText(frame, fps_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, f'Frame: {frame_count}/{total_frames}', (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            frame_path = os.path.join(self.config['temp_frames_dir'], f"frame_{frame_count:06d}.jpg")
            cv2.imwrite(frame_path, frame)
            processed_count += 1

            # if self.config['show_output']:
            #     cv2_imshow(frame)
            #     if cv2.waitKey(1) & 0xFF == ord('q'):
            #         break

        cap.release()
        self.merge_frames_to_video(frame_width, frame_height, fps)

        if processing_times:
            avg_time = np.mean(processing_times)
            print(f"\nProcessing complete!")
            print(f"Processed {processed_count} frames")
            print(f"Average processing time: {avg_time:.4f}s (~{1/avg_time:.2f} FPS)")
            print(f"Tracked {len(self.id_colors)} unique players")
            print(f"Output video saved to: {self.config['output_video_path']}")

        return True

    def merge_frames_to_video(self, width, height, fps):
        """Combine all processed frames into a video file"""
        print("\nMerging frames into video...")

        frame_files = sorted([
            f for f in os.listdir(self.config['temp_frames_dir'])
            if f.startswith('frame_') and f.endswith('.jpg')
        ])

        if not frame_files:
            print("No frames found to merge!")
            return False

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(
            self.config['output_video_path'],
            fourcc,
            fps,
            (width, height)
        )
        for frame_file in frame_files:
            frame_path = os.path.join(self.config['temp_frames_dir'], frame_file)
            frame = cv2.imread(frame_path)
            if frame is not None:
                out.write(frame)
        out.release()

        for frame_file in frame_files:
            os.remove(os.path.join(self.config['temp_frames_dir'], frame_file))
        os.rmdir(self.config['temp_frames_dir'])

        return True

if __name__ == "__main__":
    tracker = PlayerTracker(CONFIG)
    tracker.process_video()



Processing video: /content/15sec_input_720p.mp4
Resolution: 1280x720 at 25.00 FPS
Total frames: 375
Using device: cuda


Merging frames into video...

Processing complete!
Processed 375 frames
Average processing time: 0.1504s (~6.65 FPS)
Tracked 58 unique players
Output video saved to: /content/output_with_ids.mp4
